In [5]:
#下载训练集和测试集

import math
import os
#import hashlib
from urllib.request import urlretrieve
import zipfile
import gzip
import shutil
import numpy as np
from PIL import Image
from tqdm import tqdm

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras import optimizers
from keras.optimizers import SGD
import h5py


In [6]:
# 下载猫狗训练数据集并解压  
data_path = './data'  
model_path ='./models'
if not os.path.exists(model_path):
        os.makedirs(model_path)
url = 'https://storage.googleapis.com/kaggle-competitions-data/kaggle/5441/train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1521432384&Signature=EfOTgFyuNXghYaxo4e6rCkSgJUrZPvPCokVmGLotFVH%2BVywjjEId4CEZS6E%2BlHUbO2xluKaOofYYX%2FJwaSi3BFE9TD%2BjSey1BmtinJnh070m2hSi9bgW%2BV%2F6uxedBw%2FFs%2B5yD7qHMrU3eFY31hxs37osloNdw4LzhqlxLPnbdICag5p8EawoFnS%2FYtQ5Iw6BlB%2BTe4IwnSTIqK6BZQFhCfax1j8KVjAr4galJH3QfaO3ygrVb7v60olInInBpeL2prgxKfTO2rvuZABN1xOezZNGHMrw0tL7cEMy1dA%2FHgX%2Bor9P8LxbtIpMvLF1FJgucFJlwbsISiYYPXOS%2BafkxQ%3D%3D'
extract_path = os.path.join(data_path, 'train')
save_path = os.path.join(data_path, 'train.zip')
database_name ='dogs vs cats'
class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

def _unzip(save_path, _, database_name, data_path):
    print('Extracting {}...'.format(database_name))
    with zipfile.ZipFile(save_path) as zf:
        zf.extractall(data_path)
        
def download_extract(data_path, url, database_name, save_path, extract_path):
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    
    if os.path.exists(extract_path):
        print('Found {} Data'.format(database_name))
        return
        
    if not os.path.exists(save_path):
        with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Downloading {}'.format(database_name)) as pbar:
            urlretrieve(url,save_path,pbar.hook)
            
    os.makedirs(extract_path)

    try:
        print('Extracting {}...'.format(database_name))
        with zipfile.ZipFile(save_path) as zf:
            zf.extractall(data_path)
    except Exception as err:
        shutil.rmtree(extract_path)  # Remove extraction folder if there is an error
        raise err

    # Remove compressed data
    os.remove(save_path)
    print("Finished downloading and extraction")
    
download_extract(data_path, url, database_name, save_path, extract_path)

Found dogs vs cats Data


In [7]:
#将猫狗分别放进dog，cat两个文件夹
#将训练集分成训练集和验证集

train_dir = './data/train/'
validate_dir = './data/validate/'

image_files = os.listdir(train_dir)
image_files = [image_file for image_file in image_files if os.path.isfile(train_dir+image_file)]
classes = ['dog','cat']
train_dirs = [train_dir+each+'/' for each in classes]
validate_dirs = [validate_dir+each+'/' for each in classes]

flag = True
for path in train_dirs:
    if not os.path.isdir(path):
        os.makedirs(path)
        flag = False
for path in validate_dirs:
    if not os.path.isdir(path):
        os.makedirs(path)
        flag = False
if not flag:
    print("Moving images to correct folders.")
    for image_file in image_files:
        if classes[0] in image_file:
            shutil.move(train_dir+image_file, train_dirs[0]+os.path.basename(image_file))
        else:
            shutil.move(train_dir+image_file, train_dirs[1]+os.path.basename(image_file))

    for ii, image_file in enumerate(os.listdir(train_dirs[0]), 1):
        if ii >12000:
            shutil.move(train_dirs[0]+image_file, validate_dirs[0]+os.path.basename(image_file))
    for ii, image_file in enumerate(os.listdir(train_dirs[1]), 1):
        if ii >12000:
            shutil.move(train_dirs[1]+image_file, validate_dirs[1]+os.path.basename(image_file))
    print("Moving finished.")
else:
    print("Images already moved")


Images already moved


```python
data/
    train/
        dog/ ### 12000 pictures
            dog001.jpg
            dog002.jpg
            ...
        cat/ ### 12000 pictures
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dog/ ### 500 pictures
            dog001.jpg
            dog002.jpg
            ...
        cat/ ### 500 pictures
            cat001.jpg
            cat002.jpg
            ...
```


In [8]:


def get_train_and_validation_generators(img_width, img_height, train_dir, validate_dir, batch_size):
    datagen = ImageDataGenerator(rescale=1./255)

    # automagically retrieve images and their classes for train and validation sets
    train_generator = datagen.flow_from_directory(
            train_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode='binary')

    validation_generator = datagen.flow_from_directory(
            validate_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode='binary')
    return train_generator, validation_generator

def get_augmented_train_and_validation_generators(img_width, img_height, train_dir, validate_dir, batch_size):
    datagen = ImageDataGenerator(rescale=1./255)
    datagen_augmented =ImageDataGenerator(
        rescale=1./255,        # normalize pixel values to [0,1]
        shear_range=0.2,       # randomly applies shearing transformation
        zoom_range=0.2,        # randomly applies shearing transformation
        horizontal_flip=True)

    # automagically retrieve images and their classes for train and validation sets
    train_generator = datagen_augmented.flow_from_directory(
            train_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode='binary')

    validation_generator = datagen.flow_from_directory(
            validate_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode='binary')
    return train_generator, validation_generator

def get_bottleneck_train_and_validation_generators(img_width, img_height, train_dir, validate_dir, batch_size):
    datagen = ImageDataGenerator(rescale=1./255)

    # automagically retrieve images and their classes for train and validation sets
    train_generator = datagen.flow_from_directory(
            train_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode=None,
            shuffle=False)

    validation_generator = datagen.flow_from_directory(
            validate_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode=None,
            shuffle=False)
    return train_generator, validation_generator

In [13]:


def fine_tuning_model(MODEL, nb_epoch, steps_per_epoch, validation_steps,
                      train_generator, validation_generator,
                      train_optimizer1,train_optimizer2,
                      last_layers_count,dense_layer_num,model_name, 
                      img_width,img_height):
    x = Input((img_width, img_height,3))
    base_model=MODEL(input_tensor=x,weights='imagenet', include_top=False)
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(dense_layer_num, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(1, activation='sigmoid'))
    model = Sequential()
    model.add(base_model)
    model.add(top_model) 
    print(len(base_model.layers))
    print(len(top_model.layers))
           
    #train only the top layers
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer=train_optimizer1, loss='binary_crossentropy', metrics=['accuracy'])
    model.fit_generator(
        train_generator,
        epochs=nb_epoch,
        validation_data=validation_generator,
        shuffle = True)
    print('finish fine-tuning step1')
    
    # choose a few more layers to train
    layer_index = len(base_model.layers)-last_layers_count
    for layer in base_model.layers[:layer_index]:
        layer.trainable = False
    for layer in base_model.layers[layer_index:]:
        layer.trainable = True
    model.compile(optimizer=train_optimizer2, loss='binary_crossentropy', metrics=['accuracy'])
    model.fit_generator(
        train_generator,
        epochs=nb_epoch,
        validation_data=validation_generator,
        shuffle =True)

    print("finish fine-tuning step2")



In [19]:
#fine-tuning VGG16

img_width=224
img_height=224
nb_epoch = 1
batch_size =32
nb_train_samples = 24000
nb_validation_samples = 1000
MODEL=VGG16
train_optimizer1 = 'rmsprop'
train_optimizer2 = SGD(lr=0.0001, momentum=0.9)
last_layers_count =3
dense_layer_num =512
model_name='VGG16'
steps_per_epoch = nb_train_samples//batch_size
validation_steps = nb_validation_samples//batch_size

train_generator, validation_generator = get_train_and_validation_generators(img_width, img_height, 
                                                                            train_dir, validate_dir, batch_size)
fine_tuning_model(MODEL, nb_epoch, steps_per_epoch, validation_steps, 
                  train_generator, validation_generator,
                 train_optimizer1,train_optimizer2, 
                  last_layers_count,dense_layer_num,model_name,img_width,img_height)

Found 24000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
19
4
Epoch 1/1
750/750 [==============================] - 82s 110ms/step - loss: 8.0487 - acc: 0.5002 - val_loss: 8.0590 - val_acc: 0.5000
finish fine-tuning step1
Epoch 1/1
750/750 [==============================] - 82s 110ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
finish fine-tuning step2


In [15]:
#fine-tuning InceptionV3

img_width=299
img_height=299
nb_epoch = 1
batch_size =32
nb_train_samples = 24000
nb_validation_samples = 1000
MODEL=InceptionV3
train_optimizer1 = 'rmsprop'
train_optimizer2 = SGD(lr=0.0001, momentum=0.9)
last_layers_count =3
dense_layer_num =512
model_name='InceptionV3'
steps_per_epoch = nb_train_samples//batch_size
validation_steps = nb_validation_samples//batch_size

train_generator, validation_generator = get_train_and_validation_generators(img_width, img_height, 
                                                                            train_dir, validate_dir, batch_size)
fine_tuning_model(MODEL, nb_epoch, steps_per_epoch, validation_steps, 
                  train_generator, validation_generator,
                 train_optimizer1,train_optimizer2, 
                  last_layers_count,dense_layer_num,model_name,img_width,img_height)

Found 24000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
311
4
Epoch 1/1
750/750 [==============================] - 94s 126ms/step - loss: 1.5590 - acc: 0.8987 - val_loss: 0.4287 - val_acc: 0.9720
finish fine-tuning step1
Epoch 1/1
750/750 [==============================] - 93s 124ms/step - loss: 0.9899 - acc: 0.9365 - val_loss: 0.5932 - val_acc: 0.9620
finish fine-tuning step2


In [16]:
#fine-tuning Xception

img_width=299
img_height=299
nb_epoch = 1
batch_size =32
nb_train_samples = 24000
nb_validation_samples = 1000
MODEL=Xception
train_optimizer1 = 'rmsprop'
train_optimizer2 = SGD(lr=0.0001, momentum=0.9)
last_layers_count =3
dense_layer_num =512
model_name='Xception'
steps_per_epoch = nb_train_samples//batch_size
validation_steps = nb_validation_samples//batch_size

train_generator, validation_generator = get_train_and_validation_generators(img_width, img_height, 
                                                                            train_dir, validate_dir, batch_size)
fine_tuning_model(MODEL, nb_epoch, steps_per_epoch, validation_steps, 
                  train_generator, validation_generator,
                 train_optimizer1,train_optimizer2, 
                  last_layers_count,dense_layer_num,model_name,img_width,img_height)

Found 24000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
132
4
Epoch 1/1
750/750 [==============================] - 114s 152ms/step - loss: 0.7593 - acc: 0.9497 - val_loss: 0.2073 - val_acc: 0.9870
finish fine-tuning step1
Epoch 1/1
750/750 [==============================] - 112s 149ms/step - loss: 0.4443 - acc: 0.9712 - val_loss: 0.2741 - val_acc: 0.9830
finish fine-tuning step2


In [17]:
#fine-tuning InceptionResNetV2

img_width=299
img_height=299
nb_epoch = 1
batch_size =32
nb_train_samples = 24000
nb_validation_samples = 1000
MODEL=InceptionResNetV2
train_optimizer1 = 'rmsprop'
train_optimizer2 = SGD(lr=0.0001, momentum=0.9)
last_layers_count =3
dense_layer_num =512
model_name='InceptionResNetV2'
steps_per_epoch = nb_train_samples//batch_size
validation_steps = nb_validation_samples//batch_size

train_generator, validation_generator = get_train_and_validation_generators(img_width, img_height, 
                                                                            train_dir, validate_dir, batch_size)
fine_tuning_model(MODEL, nb_epoch, steps_per_epoch, validation_steps, 
                  train_generator, validation_generator,
                 train_optimizer1,train_optimizer2, 
                  last_layers_count,dense_layer_num,model_name,img_width,img_height)

Found 24000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
219062272/219055592 [==============================] - 2s 0us/step
780
4
Epoch 1/1
750/750 [==============================] - 157s 210ms/step - loss: 7.9635 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
finish fine-tuning step1
Epoch 1/1
750/750 [==============================] - 155s 206ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
finish fine-tuning step2


In [18]:
#fine-tuning ResNet50

img_width=299
img_height=299
nb_epoch = 1
batch_size =32
nb_train_samples = 24000
nb_validation_samples = 1000
MODEL=ResNet50
train_optimizer1 = 'rmsprop'
train_optimizer2 = SGD(lr=0.0001, momentum=0.9)
last_layers_count =3
dense_layer_num =512
model_name='ResNet50'
steps_per_epoch = nb_train_samples//batch_size
validation_steps = nb_validation_samples//batch_size

train_generator, validation_generator = get_train_and_validation_generators(img_width, img_height, 
                                                                            train_dir, validate_dir, batch_size)
fine_tuning_model(MODEL, nb_epoch, steps_per_epoch, validation_steps, 
                  train_generator, validation_generator,
                 train_optimizer1,train_optimizer2, 
                  last_layers_count,dense_layer_num,model_name,img_width,img_height)


Found 24000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
175
4
Epoch 1/1
750/750 [==============================] - 112s 149ms/step - loss: 0.2004 - acc: 0.9501 - val_loss: 0.0656 - val_acc: 0.9770
finish fine-tuning step1
Epoch 1/1
750/750 [==============================] - 105s 140ms/step - loss: 0.0841 - acc: 0.9747 - val_loss: 0.0403 - val_acc: 0.9830
finish fine-tuning step2
